In [1]:
import polars as pl
import os

In [2]:
path = "raw/365d/04_23-04_24"
# read all files
df = pl.read_excel(
    path + "/SouJunior_competitor_analytics_1714898767087.xlsx",
    sheet_id=1,
    read_options={ "skip_rows" : 1},
)
df

Page,Total de seguidores,Novos seguidores,Total de engajamentos da publicação,Total de publicações
str,i64,i64,i64,i64
"""SouJunior""",91364,55031,11927,500
"""Le Wagon""",77790,17935,10777,118
"""GitHub Brasil""",148705,68535,42263,224
"""Tech Pro Bem""",12555,5355,2211,30
"""Trampar na Gringa""",129922,97961,33827,356
"""VagasUX""",43473,8935,17651,285
"""Bichinhos da TI""",7819,7688,3153,60


In [3]:
def read_files(path):
    files_in_path = os.listdir(path)
    try:
        path_competitor = (
            path
            + "/"
            + [
                file
                for file in files_in_path
                if "concorrente" in file or "competitor" in file
            ][0]
        )
        path_content = (
            path
            + "/"
            + [
                file
                for file in files_in_path
                if "conteudo" in file or "content" in file or "conteúdo" in file
            ][0]
        )
        path_followers = (
            path
            + "/"
            + [
                file
                for file in files_in_path
                if "seguidor" in file or "followers" in file
            ][0]
        )
        path_visitors = (
            path
            + "/"
            + [
                file
                for file in files_in_path
                if "visitante" in file or "visitors" in file
            ][0]
        )
    except Exception as e:
        print("Erro ao encontrar arquivos de entrada. Verifique o diretório.")
        print(str(e))
        exit()

    sheets = [
        {
            "name": "content_metrics",
            "path": path_content,
            "sheet": 1,
            "skip_rows": 0,
            "super_header": True,
        },
        {
            "name": "content_posts",
            "path": path_content,
            "sheet": 2,
            "super_header": True,
        },
        {"name": "followers_new", "path": path_followers, "sheet": 1},
        {"name": "followers_location", "path": path_followers, "sheet": 2},
        {"name": "followers_function", "path": path_followers, "sheet": 3},
        {"name": "followers_experience", "path": path_followers, "sheet": 4},
        {"name": "followers_industry", "path": path_followers, "sheet": 5},
        {"name": "followers_company_size", "path": path_followers, "sheet": 6},
        {"name": "visitors_metrics", "path": path_visitors, "sheet": 1},
        {"name": "visitors_location", "path": path_visitors, "sheet": 2},
        {"name": "visitors_function", "path": path_visitors, "sheet": 3},
        {"name": "visitors_experience", "path": path_visitors, "sheet": 4},
        {"name": "visitors_industry", "path": path_visitors, "sheet": 5},
        {"name": "visitors_company_size", "path": path_visitors, "sheet": 6},
        {"name": "competitors", "path": path_competitor, "skip_rows": 1},
    ]

    for sheet in sheets:
        print("reading", sheet["name"])
        if ".xls" in sheet["path"]:
            sheet["df"] = pl.read_excel(
                source=sheet["path"],
                sheet_id=sheet.get("sheet", 1),
                read_options={"skip_rows": sheet.get("skip_rows", 0)},
                # schema_overrides={"Data": pl.Date},
            )

        else:
            sheet["df"] = pl.read_csv(source=sheet["path"])

        if "super_header" in sheet:
            first_row = sheet["df"].row(0)
            sheet["df"].columns = first_row
            sheet["df"] = sheet["df"].slice(1, sheet["df"].height)
        
        print(sheet["df"])
    return sheets


sheets = read_files(path)
df = sheets[0]["df"]
df


reading content_metrics
shape: (362, 20)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Data      ┆ Impressõe ┆ Impressõe ┆ Impressõe ┆ … ┆ Compartil ┆ Taxa de   ┆ Taxa de   ┆ Taxa de  │
│ ---       ┆ s (orgâni ┆ s (patroc ┆ s (total) ┆   ┆ hamentos  ┆ engajamen ┆ engajamen ┆ engajame │
│ str       ┆ cas)      ┆ inadas)   ┆ ---       ┆   ┆ (total)   ┆ to (orgân ┆ to (patro ┆ nto      │
│           ┆ ---       ┆ ---       ┆ str       ┆   ┆ ---       ┆ ico)      ┆ cina…     ┆ (total)  │
│           ┆ str       ┆ str       ┆           ┆   ┆ str       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 05/05/202 ┆ 2663      ┆ 57931     ┆ 60594     ┆ … ┆ 0         ┆ 0.0229064 ┆ 0.0005869 ┆ 0.001567 │
│ 3         ┆           ┆           ┆           ┆ 

Data,Impressões (orgânicas),Impressões (patrocinadas),Impressões (total),Impressões únicas (orgânicas),Cliques (orgânicos),Cliques (patrocinados),Cliques (total),Reações (orgânicas),Reações (patrocinadas),Reações (total),Comentários (orgânicos),Comentários (patrocinados),Comentários (total),Compartilhamentos (orgânicos),Compartilhamentos (patrocinados),Compartilhamentos (total),Taxa de engajamento (orgânico),Taxa de engajamento (patrocinado),Taxa de engajamento (total)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""05/05/2023""","""2663""","""57931""","""60594""","""1564""","""16""","""34""","""50""","""25""","""0""","""25""","""20""","""0""","""20""","""0""","""0""","""0""","""0.02290649643259482""","""0.0005869051112530425""","""0.0015678119945869227"""
"""05/06/2023""","""2178""","""0""","""2178""","""1195""","""62""","""0""","""62""","""4""","""0""","""4""","""0""","""0""","""0""","""0""","""0""","""0""","""0.030303030303030304""","""0""","""0.030303030303030304"""
"""05/07/2023""","""1740""","""0""","""1740""","""946""","""91""","""0""","""91""","""2""","""0""","""2""","""0""","""0""","""0""","""0""","""0""","""0""","""0.05344827586206897""","""0""","""0.05344827586206897"""
"""05/08/2023""","""3686""","""0""","""3686""","""1979""","""129""","""0""","""129""","""14""","""0""","""14""","""0""","""0""","""0""","""0""","""0""","""0""","""0.03879544221378188""","""0""","""0.03879544221378188"""
"""05/09/2023""","""2491""","""0""","""2491""","""1380""","""89""","""0""","""89""","""9""","""0""","""9""","""0""","""0""","""0""","""0""","""0""","""0""","""0.039341629867523084""","""0""","""0.039341629867523084"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""04/26/2024""","""6773""","""0""","""6773""","""4042""","""291""","""0""","""291""","""68""","""0""","""68""","""-2""","""0""","""-2""","""1""","""0""","""1""","""0.05285693193562675""","""0""","""0.05285693193562675"""
"""04/27/2024""","""2024""","""0""","""2024""","""1187""","""71""","""0""","""71""","""9""","""0""","""9""","""0""","""0""","""0""","""0""","""0""","""0""","""0.039525691699604744""","""0""","""0.039525691699604744"""
"""04/28/2024""","""1188""","""0""","""1188""","""712""","""31""","""0""","""31""","""2""","""0""","""2""","""0""","""0""","""0""","""0""","""0""","""0""","""0.027777777777777776""","""0""","""0.027777777777777776"""


In [4]:
def transform(dfs):
    for df in dfs:
        if df["name"] == "competitors":
            df["df"] = df["df"].select(df["df"].columns[:5])

        df["df"] = model_data(df["df"], category=df["name"])

        # df["df"] = transform_data(df["df"], category=df["name"])

        # if df["name"] == "content_metrics":
        #     # last_date = get_last_date(df["df"])
        #     print("Ultima data coletada - *NAO GLOBAL AINDA*", "last_date")

    return dfs


def get_last_date(df):
    return df["Date"].max().date()


def model_data(df, category):
    """
    Traduz colunas e infere tipos
    """
    print("traduzindo", category)
    category_atributes = {
        "content_metrics": {
            "Date": pl.String,  # temp
            "Impressions (organic)": pl.Int64,
            "Impressions (sponsored)": pl.Int64,
            "Impressions (total)": pl.Int64,
            "Unique impressions (organic)": pl.Int64,
            "Clicks (organic)": pl.Int64,
            "Clicks (sponsored)": pl.Int64,
            "Clicks (total)": pl.Int64,
            "Reactions (organic)": pl.Int64,
            "Reactions (sponsored)": pl.Int64,
            "Reactions (total)": pl.Int64,
            "Comments (organic)": pl.Int64,
            "Comments (sponsored)": pl.Int64,
            "Comments (total)": pl.Int64,
            "Shares (organic)": pl.Int64,
            "Shares (sponsored)": pl.Int64,
            "Shares (total)": pl.Int64,
            "Engagement rate (organic)": pl.Float64,
            "Engagement rate (sponsored)": pl.Float64,
            "Engagement rate (total)": pl.Float64,
        },
        "content_posts": {
            "Post Title": pl.String,
            "Post Link": pl.String,
            "Post Type": pl.String,
            "Campaign Name": pl.String,
            "Published by": pl.String,
            "Date": pl.String,  # temp
            "Campaign Start Date": pl.String,  # temp
            "Campaign End Date": pl.String,  # temp
            "Audience": pl.String,
            "Impressions": pl.Int64,
            "Views (excluding off-site video views)": pl.Int64,
            "Off-site Views": pl.Int64,
            "Clicks": pl.Int64,
            "Click-Through Rate (CTR)": pl.Float32,
            "Likes": pl.Int64,
            "Comments": pl.Int64,
            "Shares": pl.Int64,
            "Followers": pl.Int64,
            "Engagement Rate": pl.Float32,
            "Content Type": pl.String,
        },
        "followers_new": {
            "Date": pl.String,  # temp
            "Followers Sponsored": pl.Int64,
            "Followers Organic": pl.Int64,
            "Total Followers": pl.Int64,
        },
        "followers_location": {"Location": pl.String, "Total Followers": pl.Int64},
        "followers_function": {"Function": pl.String, "Total Followers": pl.Int64},
        "followers_experience": {
            "Experience Level": pl.String,
            "Total Followers": pl.Int64,
        },
        "followers_industry": {"Industry": pl.String, "Total Followers": pl.Int64},
        "followers_company_size": {
            "Company Size": pl.String,
            "Total Followers": pl.Int64,
        },
        "visitors_metrics": {
            "Date": pl.String,  # temp
            "Page Views Overview (Desktop)": pl.Int64,
            "Page Views Overview (Mobile Devices)": pl.Int64,
            "Page Views Overview (Total)": pl.Int64,
            "Unique Visitors Overview (Desktop)": pl.Int64,
            "Unique Visitors Overview (Mobile Devices)": pl.Int64,
            "Unique Visitors Overview (Total)": pl.Int64,
            "Page Views Day by Day (Desktop)": pl.Int64,
            "Page Views Day by Day (Mobile Devices)": pl.Int64,
            "Page Views Day by Day (Total)": pl.Int64,
            "Unique Visitors Day by Day (Desktop)": pl.Int64,
            "Unique Visitors Day by Day (Mobile Devices)": pl.Int64,
            "Unique Visitors Day by Day (Total)": pl.Int64,
            "Page Views Jobs (Desktop)": pl.Int64,
            "Page Views Jobs (Mobile Devices)": pl.Int64,
            "Page Views Jobs (Total)": pl.Int64,
            "Unique Visitors Jobs (Desktop)": pl.Int64,
            "Unique Visitors Jobs (Mobile Devices)": pl.Int64,
            "Unique Visitors Jobs (Total)": pl.Int64,
            "Total Page Views (Desktop)": pl.Int64,
            "Total Page Views (Mobile Devices)": pl.Int64,
            "Total Page Views (Total)": pl.Int64,
            "Total Unique Visitors (Desktop)": pl.Int64,
            "Total Unique Visitors (Mobile Devices)": pl.Int64,
            "Total Unique Visitors (Total)": pl.Int64,
        },
        "visitors_location": {"Location": pl.String, "Total Views": pl.Int64},
        "visitors_function": {"Function": pl.String, "Total Views": pl.Int64},
        "visitors_experience": {"Experience Level": pl.String, "Total Views": pl.Int64},
        "visitors_industry": {"Industry": pl.String, "Total Views": pl.Int64},
        "visitors_company_size": {"Company Size": pl.String, "Total Views": pl.Int64},
        "competitors": {
            "Page": pl.String,
            "Total Followers": pl.Int64,
            "New Followers": pl.Int64,
            "Total Post Engagements": pl.Float32,
            "Total Posts": pl.Int64,
        },
    }
    category_atribute = category_atributes.get(category)

    translated_columns = list(category_atribute.keys())
    df.columns = translated_columns

    df = df.cast(category_atribute)

    date_columns = {
        "content_metrics": ["Date"],
        "content_posts": ["Date", "Campaign Start Date", "Campaign End Date"],
        "followers_new": ["Date"],
        "visitors_metrics": ["Date"],
    }

    if category in date_columns.keys():
        for col in date_columns[category]:
            df = df.with_columns(pl.col(col).str.to_date("%m/%d/%Y"))

    return df


def transform_data(df, category):
    if category == "content_metrics":
        df = df[
            [
                "Date",
                "Impressions (total)",
                "Clicks (total)",
                "Reactions (total)",
                "Comments (total)",
                "Shares (total)",
                "Engagement rate (total)",
            ]
        ]
        # # # # # # # # # # df = df.select(
        # # # # # # # # # #     (pl.col("nrs") + 5).alias("Reactions (positive)"),
        # # # # # # # # # #     (pl.col("nrs") - 5).alias("nrs - 5"),
        # # # # # # # # # # )

        df["Reactions (positive)"] = df["Reactions (total)"][
            df["Reactions (total)"] >= 0
        ]
        df["Comments (positive)"] = df["Comments (total)"][df["Comments (total)"] >= 0]
        df["Shares (positive)"] = df["Shares (total)"][df["Shares (total)"] >= 0]
        df["Clicks (positive)"] = df["Clicks (total)"][df["Clicks (total)"] >= 0]

        df["Reactions (positive)"] = df["Reactions (positive)"].fillna(0)
        df["Comments (positive)"] = df["Comments (positive)"].fillna(0)
        df["Shares (positive)"] = df["Shares (positive)"].fillna(0)
        df["Clicks (positive)"] = df["Clicks (positive)"].fillna(0)

        window = 3

        df["Reactions (moving average)"] = (
            df["Reactions (positive)"].rolling(window=window).mean()
        )
        df["Comments (moving average)"] = (
            df["Comments (positive)"].rolling(window=window).mean()
        )
        df["Shares (moving average)"] = (
            df["Shares (positive)"].rolling(window=window).mean()
        )
        df["Clicks (moving average)"] = (
            df["Clicks (positive)"].rolling(window=window).mean()
        )

        df["Reactions"] = df.apply(
            lambda row: (
                row["Reactions (moving average)"]
                if row["Reactions"] < 0
                else row["Reactions"]
            ),
            axis=1,
        )

        df["Comments"] = df.apply(
            lambda row: (
                row["Comments (moving average)"]
                if row["Comments"] < 0
                else row["Comments"]
            ),
            axis=1,
        )

        df["Shares"] = df.apply(
            lambda row: (
                row["Shares (moving average)"] if row["Shares"] < 0 else row["Shares"]
            ),
            axis=1,
        )

        df["Clicks"] = df.apply(
            lambda row: (
                row["Clicks (moving average)"] if row["Clicks"] < 0 else row["Clicks"]
            ),
            axis=1,
        )

        df["Engagement Rate"] = df.apply(
            lambda row: (
                row["Reactions"] + row["Comments"] + row["Clicks"] + row["Shares"]
            )
            / row["Impressions"],
            axis=1,
        )

        df = df[
            [
                "Date",
                "Impressions",
                "Clicks",
                "Reactions",
                "Comments",
                "Shares",
                "Engagement Rate",
            ]
        ]

    if "Date" in df.columns:
        df["Date"] = pl.to_datetime(df["Date"])

    else:
        print("temp")
        df.insert(0, "Date", "12/12/2021")

    return df

In [5]:
modeled_data = transform(sheets)

traduzindo content_metrics
traduzindo content_posts
traduzindo followers_new
traduzindo followers_location
traduzindo followers_function
traduzindo followers_experience
traduzindo followers_industry
traduzindo followers_company_size
traduzindo visitors_metrics
traduzindo visitors_location
traduzindo visitors_function
traduzindo visitors_experience
traduzindo visitors_industry
traduzindo visitors_company_size
traduzindo competitors


In [6]:
transform_data(sheets[0]["df"], "content_metrics")

TypeError: cannot use `__getitem__` on Series of dtype Int64 with argument shape: (362,)
Series: 'Reactions (total)' [bool]
[
	true
	true
	true
	true
	true
	…
	true
	true
	true
	true
	true
] of type 'Series'

In [7]:
content_metrics = sheets[0]["df"]
df = content_metrics
df = df[
    [
        "Date",
        "Impressions (total)",
        "Clicks (total)",
        "Reactions (total)",
        "Comments (total)",
        "Shares (total)",
        "Engagement rate (total)",
    ]
]
# df = df.with_columns(
#     pl.sum("nrs").alias("nrs_sum"),
#     pl.col("random").count().alias("count"),
# )
df
df = df.with_columns(
    pl.when(pl.col("Reactions (total)") >= 0)
    .then(pl.col("Reactions (total)"))
    .otherwise(pl.lit(0))
    .alias("Reactions (positive)"),
    pl.when(pl.col("Comments (total)") >= 0)
    .then(pl.col("Comments (total)"))
    .otherwise(pl.lit(0))
    .alias("Comments (positive)"),
    pl.when(pl.col("Shares (total)") >= 0)
    .then(pl.col("Shares (total)"))
    .otherwise(pl.lit(0))
    .alias("Shares (positive)"),
    pl.when(pl.col("Clicks (total)") >= 0)
    .then(pl.col("Clicks (total)"))
    .otherwise(pl.lit(0))
    .alias("Clicks (positive)"),
)

window = 3

df = df.with_columns(
    (pl.col("Reactions (positive)"))
    .rolling_mean(window_size=3)
    .alias("Reactions (moving average)"),
    (pl.col("Comments (positive)"))
    .rolling_mean(window_size=3)
    .alias("Comments (moving average)"),
    (pl.col("Shares (positive)"))
    .rolling_mean(window_size=3)
    .alias("Shares (moving average)"),
    (pl.col("Clicks (positive)"))
    .rolling_mean(window_size=3)
    .alias("Clicks (moving average)"),
)

df = df.with_columns(
    pl.when(pl.col("Reactions (total)") >= 0)
    .then(pl.col("Reactions (total)"))
    .otherwise("Reactions (moving average)")
    .alias("Reactions (total 2)"),
    pl.when(pl.col("Comments (total)") >= 0)
    .then(pl.col("Comments (total)"))
    .otherwise("Comments (moving average)")
    .alias("Comments (total 2)"),
    pl.when(pl.col("Shares (total)") >= 0)
    .then(pl.col("Shares (total)"))
    .otherwise("Shares (moving average)")
    .alias("Shares (total 2)"),
    pl.when(pl.col("Clicks (total)") >= 0)
    .then(pl.col("Clicks (total)"))
    .otherwise("Clicks (moving average)")
    .alias("Clicks (total 2)"),
)

engagement_sum = (
    pl.col("Reactions (total 2)")
    + pl.col("Comments (total 2)")
    + pl.col("Clicks (total 2)")
    + pl.col("Shares (total 2)")
)

df = df.with_columns(engagement_sum / pl.col("Impressions (total)").alias("XXXXXXXX"))
df

Date,Impressions (total),Clicks (total),Reactions (total),Comments (total),Shares (total),Engagement rate (total),Reactions (positive),Comments (positive),Shares (positive),Clicks (positive),Reactions (moving average),Comments (moving average),Shares (moving average),Clicks (moving average),Reactions (total 2),Comments (total 2),Shares (total 2),Clicks (total 2)
date,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-05,60594,50,25,20,0,0.001568,25,20,0,50,null,null,null,null,0.001568,20.0,0.0,50.0
2023-05-06,2178,62,4,0,0,0.030303,4,0,0,62,null,null,null,null,0.030303,0.0,0.0,62.0
2023-05-07,1740,91,2,0,0,0.053448,2,0,0,91,10.333333,6.666667,0.0,67.666667,0.053448,0.0,0.0,91.0
2023-05-08,3686,129,14,0,0,0.038795,14,0,0,129,6.666667,0.0,0.0,94.0,0.038795,0.0,0.0,129.0
2023-05-09,2491,89,9,0,0,0.039342,9,0,0,89,8.333333,0.0,0.0,103.0,0.039342,0.0,0.0,89.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-04-26,6773,291,68,-2,1,0.052857,68,0,1,291,46.666667,4.666667,0.666667,350.0,0.053841,4.666667,1.0,291.0
2024-04-27,2024,71,9,0,0,0.039526,9,0,0,71,42.0,1.666667,0.666667,334.333333,0.039526,0.0,0.0,71.0
2024-04-28,1188,31,2,0,0,0.027778,2,0,0,31,26.333333,0.0,0.333333,131.0,0.027778,0.0,0.0,31.0


: 

In [ ]:
cols = [
    "Date",
    "Impressions (organic)",
    "Impressions (sponsored)",
    "Impressions",
    "Unique impressions (organic)",
    "Clicks (organic)",
    "Clicks (sponsored)",
    "Clicks",
    "Reactions (organic)",
    "Reactions (sponsored)",
    "Reactions",
    "Comments (organic)",
    "Comments (sponsored)",
    "Comments",
    "Shares (organic)",
    "Shares (sponsored)",
    "Shares",
    "Engagement rate (organic)",
    "Engagement rate (sponsored)",
    "Engagement rate",
]
# content_metrics =
out = sheets[0]["df"].select(
    pl.col("Date").cast(pl.Date).alias("Date"),
    pl.col("Impressions").cast(pl.Int32).alias("Impressions"),
    pl.col("Clicks").cast(pl.Int32).alias("Clicks"),
    pl.col("Reactions").cast(pl.Int32).alias("Reactions"),
    pl.col("Comments").cast(pl.Int32).alias("Comments"),
    pl.col("Shares").cast(pl.Int32).alias("Shares"),
    pl.col("Engagement rate").cast(pl.Float32).alias("Engagement rate"),
)

out.with_column((pl.col("Impressions") * pl.col("Reactions")).alias("teste"))
# sheets[0]["df"]

ColumnNotFoundError: Date